# Experiments on the Brazil subset from Denv4 data set

In what follows, we will compare four methods for non-bifurcating phylogenetic inference: 

* `Maximum Likelihood (ML) + bootstrap`
* `reversible jump MCMC (rjMCMC)`
* `ML + bootstrap + threshold`
* `ML + bootstrap + multistep adaptive LASSO` 

on the Brazil subset of the Denv4 data set.

__Note__: please change the path to the outputs of different methods accordingly when read in the results. 

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%gui tk

import sys
sys.path.append('../')
import numpy as np
from time import clock
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import re
import commands

In [3]:
import phyloinfer as pinf
from model import PHY
from optimizer import adaLasso, raxml
from utils import readTree, saveTree, treePosterior, addLabel, maptaxonname, Collapse, splitSupport

from p4 import *
import dendropy
from Bio import Phylo
from collections import defaultdict
from cStringIO import StringIO

## Load the data and preprocess MrBayes output

In [29]:
denv4_brazil_seq = '../data/Dengue/dengue_denv4_aligned_brazil.fasta'

# preprocess MCMC sampels from MrBayes
tree_list_mcmc = dendropy.TreeList.get(path='../data/Dengue/dengue_denv4_aligned_brazil.nexus.t',
                                       schema='nexus', rooting='force-unrooted')
_, denv4_brazil_taxa = pinf.data.loadData(denv4_brazil_seq, data_type='fasta')


# transform to p4 compatible format
addLabel(tree_list_mcmc)
taxon_names = tree_list_mcmc.taxon_namespace
maptaxonname(taxon_names, denv4_brazil_taxa)

tree_list_mcmc.write(path='../data/Dengue/dengue_denv4_aligned_brazil.nexus.t.transformed',
                     schema='nexus', suppress_taxa_blocks=True, translate_tree_taxa=True)

## Run rjMCMC in p4

In [7]:
# load data
read(denv4_brazil_seq)
denv4_brazil_data = Data()
denv4_brazil_taxa = denv4_brazil_data.taxNames

In [8]:
# step up rjmcmc
init_tree = func.randomTree(taxNames=denv4_brazil_taxa)
init_tree.data = denv4_brazil_data

init_tree.newComp(free=0, spec='equal')
init_tree.newRMatrix(free=0, spec='ones')
init_tree.setNGammaCat(nGammaCat=1)
init_tree.setPInvar(free=0, val=0.0)

In [ ]:
# run reversible jump mcmc
m = Mcmc(init_tree, nChains=4, runNum=0, sampleInterval=1000, checkPointInterval=2500000)
m.prob.polytomy = 1.0
m.prob.brLen = 0.1
m.prob.local = 0.1
m.prob.eTBR = 0.1
m.tunings.chainTemp = 0.0

m.run(10000000)

In [30]:
# get the mcmc consensus tree
tp = TreePartitions('../data/Dengue/dengue_denv4_aligned_brazil.nexus.t.transformed', skip=2000)
majority_tree = tp.consensus()

node_idx = 0
for n in majority_tree.iterInternalsNoRoot():
    node_idx += 1
    n.name = "({}):{:.0f}".format(node_idx, n.br.support*100)

# print the mcmc consensus tree
majority_tree.eps('../results/consensus_mcmc.eps', width=800, putInternalNodeNamesOnBranches=1)

In [31]:
# make a dictionary of a split key on the mcmc consensus tree.
mcmc_consensus_dict = {}
majority_tree.makeSplitKeys()
for n in majority_tree.iterInternalsNoRoot():
    mcmc_consensus_dict[n.br.splitKey] = n

In [32]:
# aggregate result into a split table.
splitTable = np.array([[int(re.findall('\d+',n.name)[0]), int('{:.0f}'.format(n.br.support*100))] for splitKey, n in sorted(mcmc_consensus_dict.items(),key=lambda x:x[0])])
splitTable = splitTable[splitTable[:,0].argsort()]

# note: please change the path to match your rjmcmc outputs or vice versa
read('../results/mcmc_trees_0.nex')
rjmcmc_split_dict = splitSupport(var.trees, mcmc_consensus_dict, denv4_brazil_taxa, skip=0.2)
rjmcmc_split = np.array([[int(re.findall('\d+',mcmc_consensus_dict[key].name)[0]), percent] for key, percent in sorted(rjmcmc_split_dict.items(), key=lambda x:x[0])])
rjmcmc_split = rjmcmc_split[rjmcmc_split[:,0].argsort()][:,1]
rjmcmc_split = rjmcmc_split.reshape((len(rjmcmc_split),1)) 

splitTable = np.concatenate((splitTable,rjmcmc_split), axis=1)

## Run raxml for maximum likelihood bootstrapping

In [ ]:
# run raxml (take some time)
clear = commands.getoutput('rm '+denv4_brazil_seq+'.raxml.*')
raxml(sequences=denv4_brazil_seq, model="JC", starting_tree="pars{10}", bootstrap=True, bstrees=1000, log=True)

In [26]:
# transform to p4 compatible format.  
tree_list = dendropy.TreeList.get(path='../data/Dengue/dengue_denv4_aligned_brazil.fasta.raxml.bootstraps', schema='newick', rooting="force-unrooted")
addLabel(tree_list)
tree_list.write(path='../data/Dengue/dengue_denv4_aligned_brazil.fasta.raxml.bootstraps.nexus', schema='nexus', suppress_taxa_blocks=True, translate_tree_taxa=True, unquoted_underscores=True)

In [33]:
# read bootstrap trees into p4
var.trees = []
read('../data/Dengue/dengue_denv4_aligned_brazil.fasta.raxml.bootstraps.nexus')
ml_bs_dict = splitSupport(var.trees, mcmc_consensus_dict, denv4_brazil_taxa, skip=0.0)
ml_bs_split = np.array([[int(re.findall('\d+',mcmc_consensus_dict[key].name)[0]), percent] for key, percent in sorted(ml_bs_dict.items(), key=lambda x:x[0])])
ml_bs_split = ml_bs_split[ml_bs_split[:,0].argsort()][:,1]
ml_bs_split = ml_bs_split.reshape((len(ml_bs_split),1))

splitTable = np.concatenate((splitTable,ml_bs_split), axis=1)
splitTable[:,[1,2,3]] = splitTable[:,[3,1,2]]

## ML + bootstrap + threshold

In [13]:
thresholds = [1e-06, 5e-05, 1e-04]

for threshold in thresholds:
    ml_bs_threshold_dict = splitSupport(var.trees, mcmc_consensus_dict, denv4_brazil_taxa, collapse_threshold=threshold, skip=0.0)
    ml_bs_threshold_split = np.array([[int(re.findall('\d+',mcmc_consensus_dict[key].name)[0]), percent] for key, percent in sorted(ml_bs_threshold_dict.items(), key=lambda x:x[0])])
    ml_bs_threshold_split = ml_bs_threshold_split[ml_bs_threshold_split[:,0].argsort()][:,1]
    ml_bs_threshold_split = ml_bs_threshold_split.reshape((len(ml_bs_threshold_split),1))
    
    splitTable = np.concatenate((splitTable,ml_bs_threshold_split), axis=1)

## ML + bootstrap + multistep adaptive LASSO

In [40]:
# set up the model
pden = np.array([0.25, 0.25, 0.25, 0.25])
D, U, U_inv, rate_matrix = pinf.rateM.decompJC()

# load data
denv4_brazil, _ = pinf.data.loadData(denv4_brazil_seq, 'fasta')
model_denv4_brazil = PHY(pden, ('JC', 1.0), denv4_brazil)

In [ ]:
# run multistep adaptive LASSO

# read bootstrap trees
bs_trees = readTree('../data/Dengue/dengue_denv4_aligned_brazil.fasta.raxml.bootstraps', tree_format=5)

brlen_ada_list = []
ll_ada_list = []
ada_time_phy_list = []

lam = 1e-07
beta = 0.5
gamma_ada_penalized = 1.0
gamma_phy_list = [150, 300, 450]

for gamma_phy in gamma_phy_list:
    bs_trees_copy = []
    for tree in bs_trees:
        tree_copy = deepcopy(tree)
        pinf.tree.namenum(tree_copy, denv4_brazil_taxa)
        brlen_ml_bs_denv4_brazil = pinf.branch.get(tree_copy)

        brlen_init = np.copy(brlen_ml_bs_denv4_brazil)

        start = clock()
        brlen_ada_lasso, objval_ll_ada_lasso, objval_lasso_ada_lasso, n_zeros_ada_lasso, lam_tuned_ada_lasso = adaLasso(model_denv4_brazil, tree_copy, brlen_init, lam, gamma=gamma_phy, beta=beta, prox='l1', msteps=3, gamma_ada_penalized=gamma_ada_penalized, minstepsz=1e-09, sparsity_monitor=True)

        brlen_ada_list.append(brlen_ada_lasso[-1])
        ll_ada_list.append(objval_ll_ada_lasso[-1])
        ada_time_phy_list.append(clock() - start)
        pinf.branch.set(tree_copy, brlen_ada_lasso[-1])
        pinf.tree.nametaxon(tree_copy, denv4_brazil_taxa)
        bs_trees_copy.append(tree_copy)

        print "\nlambda = {}; step size: {}; elasped time: {:.04f} seconds".format(gamma_phy, lam_tuned_ada_lasso, ada_time_phy_list[-1])

        plt.plot(objval_ll_ada_lasso, label="LL")
        plt.plot(objval_lasso_ada_lasso, label="LL+Penalty")
        plt.legend(loc=4)
        plt.show()
    
    saveTree(bs_trees_copy, '../data/Dengue/dengue_denv4_aligned_brazil_gamma{}_10.fasta.raxml.bootstraps.adalasso'.format(gamma_phy))

In [17]:
for gamma_phy in gamma_phy_list:
    # transform to p4 compatible format
    tree_list_bs_adalasso = dendropy.TreeList.get(path='../data/Dengue/dengue_denv4_aligned_brazil_gamma{}_10.fasta.raxml.bootstraps.adalasso'.format(gamma_phy), schema='newick', rooting="force-unrooted")
    addLabel(tree_list_bs_adalasso)
    tree_list_bs_adalasso.write(path='../data/Dengue/dengue_denv4_aligned_brazil_gamma{}_10.fasta.raxml.bootstraps.adalasso.nexus'.format(gamma_phy), schema='nexus', suppress_taxa_blocks=True, translate_tree_taxa=True, unquoted_underscores=True)

    var.trees = []
    read('../data/Dengue/dengue_denv4_aligned_brazil_gamma{}_10.fasta.raxml.bootstraps.adalasso.nexus'.format(gamma_phy))
    
    # collapse zero branches
    for tree in var.trees:
        Collapse(tree, 0.0)
    
    ml_bs_adalasso_dict = splitSupport(var.trees, mcmc_consensus_dict, denv4_brazil_taxa, collapse_threshold=0.0, skip=0.0)
    ml_bs_adalasso_split = np.array([[int(re.findall('\d+',mcmc_consensus_dict[key].name)[0]), percent] for key, percent in sorted(ml_bs_adalasso_dict.items(), key=lambda x:x[0])])
    ml_bs_adalasso_split = ml_bs_adalasso_split[ml_bs_adalasso_split[:,0].argsort()][:,1]
    ml_bs_adalasso_split = ml_bs_adalasso_split.reshape((len(ml_bs_adalasso_split),1))
    splitTable = np.concatenate((splitTable,ml_bs_adalasso_split), axis=1)

In [ ]:
# show split support Table
splitTable